<a href="https://colab.research.google.com/github/aleschiavo94/Sketches-Web-Search-Engine/blob/main/code/mAP_by_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import numpy as np
import time
import itertools
import os, shutil
import tensorflow as tf
import matplotlib.pyplot as plt

from random import random
from ipywidgets import Image
from numpy.linalg import norm
from IPython.display import display
from keras.models import Model
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping, ModelCheckpoint

# tensorflow version 2.4.0
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, InceptionV3

In [ ]:
zip_path = '/content/gdrive/Shareddrives/COMPUTER_VISION/MIM_zipped.zip'
!cp "{zip_path}" .
!unzip -q MIM_zipped.zip
!rm MIM_zipped.zip

In [ ]:
test_images_per_class = 20 # le immagini sono al massimo 80
train_images_per_class = 80 - test_images_per_class

In [ ]:
BASE_DIR = "/content/content/gdrive/Shareddrives/COMPUTER_VISION/MIRCV"
SKETCHES_DIR = os.path.join(BASE_DIR, "sketches")
MIRFLICKR_DIR = os.path.join(BASE_DIR, "mirflickr/mirflickr25k")

print(os.listdir(os.path.join(SKETCHES_DIR, "png")))
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)
      
copytree(f'{SKETCHES_DIR}/png', f'{SKETCHES_DIR}/all')

try:
  os.rename(f'{SKETCHES_DIR}/png', f'{SKETCHES_DIR}/train')
except:
  print('already renamed')

if not os.path.exists(f'{SKETCHES_DIR}/test'):
  os.mkdir(f'{SKETCHES_DIR}/test')
  for class_folder in os.listdir(f'{SKETCHES_DIR}/train'):
    if os.path.isdir(f'{SKETCHES_DIR}/train/{class_folder}'):
      images = os.listdir(f'{SKETCHES_DIR}/train/{class_folder}')
      os.mkdir(f'{SKETCHES_DIR}/test/{class_folder}')
      for i in range(test_images_per_class):
        os.rename(f'{SKETCHES_DIR}/train/{class_folder}/{images[i]}', f'{SKETCHES_DIR}/test/{class_folder}/{images[i]}')

['tractor', 'trombone', 'megaphone', 'microphone', 'wine-bottle', 'foot', 'shoe', 'pretzel', 'leaf', 'house', 'sailboat', 'hot air balloon', 't-shirt', 'piano', 'octopus', 'mermaid', 'motorbike', 'pipe (for smoking)', 'revolver', 'rainbow', 'pigeon', 'vase', 'stapler', 'mouse (animal)', 'scissors', 'rifle', 'umbrella', 'panda', 'potted plant', 'teddy-bear', 'grenade', 'brain', 'bush', 'snake', 'hourglass', 'train', 'giraffe', 'spoon', 'cup', 'comb', 'fire hydrant', 'sheep', 'mushroom', 'pumpkin', 'socks', 'moon', 'computer-mouse', 'mouth', 'standing bird', 'speed-boat', 'key', 'pineapple', 'candle', 'carrot', 'cat', 'flower with stem', 'santa claus', 'suitcase', 'hedgehog', 'mug', 'pizza', 'flying saucer', 'snowman', 'scorpion', 'cactus', 'apple', 'wrist-watch', 'seagull', 'power outlet', 'bowl', 'squirrel', 'blimp', 'feather', 'race car', 'tablelamp', 'camel', 'binoculars', 'laptop', 'cabinet', 'rabbit', 'nose', 'loudspeaker', 'keyboard', 'strawberry', 'ant', 'bicycle', 'hamburger', '

In [ ]:
class NO_INDEX:
  def __init__(self):
    self.features = None
    self.ids = None
    self.labels = None

  def insert(self, features, ids, labels):
    assert features.shape[0] == len(ids), "mismatch between ids length and features"
    assert len(labels) == len(ids), "mismatch between ids length and labels"
    self.features = features
    self.ids = ids
    self.labels = labels
  
  def query(self, features, top_k, mode = 'euclidean', return_cost = False):
    assert mode in ['similarity', 'euclidean'], "mode must be similarity or euclidean"
    top_k += 1
    if mode == 'euclidean':
      dist = norm(self.features - np.array(features), axis=1)
      idx_partitioned = np.argpartition(dist, top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0] - 1)
      k = {}
      # qua è più comodo avere array 1- dimensionali
      k['ids'] = self.ids[idx_partitioned].flatten()[:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      k['labels'] = self.labels[idx_partitioned].flatten()[:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      k['distances'] = dist[idx_partitioned][:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      
      idx_sorted = np.argsort(k['distances'])
      idx_sorted = idx_sorted
      k['distances'] = k['distances'][idx_sorted]
      k['ids'] = k['ids'][idx_sorted]
      k['labels'] = k['labels'][idx_sorted]
      if return_cost:
        return (k, dist.size)
      return k
    sim = np.sum(self.features * np.array(features), axis=1) / (norm(self.features, axis=1) * norm(np.array([features]), axis=1))
    idx_partitioned = np.argpartition(sim, -(top_k - 1) if sim.shape[0] - 1 > top_k - 1 else sim.shape[0] - 1)
    k = {}
    # qua è più comodo avere array 1- dimensionali
    if sim.shape[0] - 1 > top_k - 1:  
      idx_partitioned = idx_partitioned[-(top_k - 1):]

    k['ids'] = self.ids[idx_partitioned].flatten()
    k['labels'] = self.labels[idx_partitioned].flatten()
    k['similarities'] = sim[idx_partitioned]
    idx_sorted = np.argsort(k['similarities'])[::-1]
    idx_sorted = idx_sorted
    k['similarities'] = k['similarities'][idx_sorted]
    k['ids'] = k['ids'][idx_sorted]
    k['labels'] = k['labels'][idx_sorted]
    if return_cost:
      return (k, sim.size)
    return k
    

In [ ]:
IMG_HEIGHT = 299
IMG_WIDTH = 299
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
BATCH_SIZE = 64

In [ ]:
#yields batches of images from the subdirectories 
#found in the sketches directory [class_0 ... class_250]
#together with class labels.

#images normalization 
sketches_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
mirflickr_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

sketches_generator = sketches_datagen.flow_from_directory(
        # This is the target directory
        SKETCHES_DIR + '/all',
        shuffle=False,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE)

mirflickr_generator = mirflickr_datagen.flow_from_directory(
        # This is the target directory
        MIRFLICKR_DIR,
        shuffle=False,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE)

Found 20000 images belonging to 250 classes.
Found 25000 images belonging to 1 classes.


In [ ]:
#Extracting features using the pretrained convolutional base 

def extract_features(extractor, generator, sample_count, dim=2048):
  features = np.zeros((sample_count, dim)) #extractor output shape 
  i = 0
  for inputs_batch, labels_batch in generator:
    start = time.time()
    features_batch = extractor.predict(inputs_batch)
    start = time.time()
    if (i + 1) * BATCH_SIZE > sample_count:
      features[i * BATCH_SIZE : sample_count , :] = features_batch
      assert np.array_equal(np.argmax(labels_batch, axis = 1), generator.labels[i * BATCH_SIZE : sample_count]), 'LABELS NOT CORRESPONDING REINIZIALIZE GENERATOR'
    else:
      features[i * BATCH_SIZE : (i + 1) * BATCH_SIZE, : ] = features_batch
      assert np.array_equal(np.argmax(labels_batch, axis = 1), generator.labels[i * BATCH_SIZE : (i + 1) * BATCH_SIZE]), 'LABELS NOT CORRESPONDING REINIZIALIZE GENERATOR'
    i += 1
    if i * BATCH_SIZE >= sample_count:
      break
  
  return features


In [ ]:
MODEL_PATH = "/content/gdrive/Shareddrives/COMPUTER_VISION/models"

In [ ]:
# BEST MODEL -> inception_finetuning_classification_3_last_one_more_train_all_parameters_more_train.h5
loaded = models.load_model(os.path.join(MODEL_PATH, 'inception_finetuning_classification_3_last_one_more_train_all_parameters_more_train.h5'))
#loaded.summary()

finetuned_extactor = Model(loaded.input, loaded.layers[-2].output)
finetuned_extactor.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_376 (Conv2D)             (None, None, None, 3 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_376 (BatchN (None, None, None, 3 96          conv2d_376[0][0]                 
__________________________________________________________________________________________________
activation_376 (Activation)     (None, None, None, 3 0           batch_normalization_376[0][0]    
______________________________________________________________________________________________

In [ ]:
sketches_features_finetuned = extract_features(finetuned_extactor, sketches_generator, 20000)
mirflickr_features_finetuned = extract_features(finetuned_extactor, mirflickr_generator, 25000)

In [ ]:
def average_precision(requested_label, result_labels, n_ground_truth = 80):
  """
  label ricercata, label ottenute, il numero di oggetti che ci sono quella label
  """
  correct_array = (requested_label == result_labels).astype(int)
  precision_array = [np.mean(correct_array[:k]) for k in range(1, correct_array.shape[0] + 1)]
  # print(precision_array * correct_array) # mi rimangono solo quelli a 1
  return np.sum(precision_array * correct_array) / n_ground_truth


In [ ]:
"""
per ogni class label faccio 80 queries (usando ogni volta come oggetto query uno degli 80 oggetti di quella classe)
     per ogni query mi calcolo l' AP
     calcolo la mAP. mix(AP), max(AP), std(AP), range(AP)
"""
def get_class_AP_info(index, features, n_queries = 80, n_labels = 250, img_per_labels = 80, mode = 'similarity'):
  counter = 0
  classes_AP = []
  for label in range(n_labels): 
    AP_values = []
    counter += 1
    for i in range(img_per_labels):
      image_idx = label*n_queries + i 

      #print(image_idx)
      #if label*n_queries >= 80:
      #  # sketch image displaying 
      #  image1 = SKETCHES_DIR + '/png/alarm clock/' + str(image_idx+1) + ".png"
      #  display(Image.from_file(image1, width=229, height=229))

      res = index.query(features[image_idx], img_per_labels, mode = mode)
      assert res['labels'][0] == label, 'deve essere della stessa label'
      a = average_precision(label, res['labels'], img_per_labels)
    
      AP_values.append(a)
    classes_AP.append(AP_values)
    if counter == 100:
      return classes_AP

In [ ]:
no_index_finetuned = NO_INDEX()
no_index_finetuned.insert(np.vstack((sketches_features_finetuned, mirflickr_features_finetuned)), 
                np.concatenate((sketches_generator.filenames, mirflickr_generator.filenames)), 
                np.concatenate((sketches_generator.labels, np.array([250] * mirflickr_features_finetuned.shape[0]))))

In [ ]:
classes_AP = get_class_AP_info(no_index_finetuned, sketches_features_finetuned)
#print(classes_AP)

In [ ]:
class_names = os.listdir(os.path.join(SKETCHES_DIR, "all"))
class_names.remove("filelist.txt")
class_names = sorted(class_names)
print(class_names)
print(len(class_names))

['airplane', 'alarm clock', 'angel', 'ant', 'apple', 'arm', 'armchair', 'ashtray', 'axe', 'backpack', 'banana', 'barn', 'baseball bat', 'basket', 'bathtub', 'bear (animal)', 'bed', 'bee', 'beer-mug', 'bell', 'bench', 'bicycle', 'binoculars', 'blimp', 'book', 'bookshelf', 'boomerang', 'bottle opener', 'bowl', 'brain', 'bread', 'bridge', 'bulldozer', 'bus', 'bush', 'butterfly', 'cabinet', 'cactus', 'cake', 'calculator', 'camel', 'camera', 'candle', 'cannon', 'canoe', 'car (sedan)', 'carrot', 'castle', 'cat', 'cell phone', 'chair', 'chandelier', 'church', 'cigarette', 'cloud', 'comb', 'computer monitor', 'computer-mouse', 'couch', 'cow', 'crab', 'crane (machine)', 'crocodile', 'crown', 'cup', 'diamond', 'dog', 'dolphin', 'donut', 'door', 'door handle', 'dragon', 'duck', 'ear', 'elephant', 'envelope', 'eye', 'eyeglasses', 'face', 'fan', 'feather', 'fire hydrant', 'fish', 'flashlight', 'floor lamp', 'flower with stem', 'flying bird', 'flying saucer', 'foot', 'fork', 'frog', 'frying-pan', 'g

In [ ]:
import ipywidgets as widgets
import IPython.display as display

classes_stats = []

for label, item in enumerate(classes_AP):
  tmp = {} 

  #get max value and max value index 
  max_elem = max(item)
  max_index = [i for i, j in enumerate(item) if j == max_elem]
  max_index = label*80 + max_index[0] + 1

  #get min value and min value index 
  min_elem = np.min(item)
  min_index = [i for i, j in enumerate(item) if j == min_elem]
  min_index = label*80 + min_index[0] + 1

  #max min range
  max_min_range = max(item) - min(item)

  #standard deviation 
  std = np.std(item)

  print(f"--------------------------- Class {label}: {class_names[label]} --------------------------- ")
  print(f"Min AP: {round(min_elem, 4)}  Sketch Index: {min_index} | Max AP: {round(max_elem, 4)}  Sketch Index: {max_index}")

  # show worst & best class sketches side by side
  image1 = SKETCHES_DIR + '/all/' + str(class_names[label]) + '/' + str(min_index) + ".png"
  image2 = SKETCHES_DIR + '/all/' + str(class_names[label]) + '/' + str(max_index) + ".png"
  img1 = open(image1, 'rb').read()
  img2 = open(image2, 'rb').read()
  wi1 = widgets.Image(value=img1, format='png', width=229, height=229)
  wi2 = widgets.Image(value=img2, format='png', width=229, height=229)
  sidebyside = widgets.HBox([wi1, wi2])
  display.display(sidebyside)

  tmp["class"] = class_names[label]
  tmp["label"] = label
  tmp["mAP"] = np.mean(item)
  tmp["min AP"] = min_elem
  tmp["max AP"] = max_elem
  tmp["max-min range"] = max_min_range
  tmp["std"] = std
  
  classes_stats.append(tmp)
  #print(tmp)

--------------------------- Class 0: airplane --------------------------- 
Min AP: 0.0125  Sketch Index: 7 | Max AP: 0.7779  Sketch Index: 13


--------------------------- Class 1: alarm clock --------------------------- 
Min AP: 0.0125  Sketch Index: 116 | Max AP: 0.9018  Sketch Index: 97


--------------------------- Class 2: angel --------------------------- 
Min AP: 0.0125  Sketch Index: 164 | Max AP: 0.6902  Sketch Index: 230


--------------------------- Class 3: ant --------------------------- 
Min AP: 0.0125  Sketch Index: 241 | Max AP: 0.7913  Sketch Index: 252


--------------------------- Class 4: apple --------------------------- 
Min AP: 0.0787  Sketch Index: 391 | Max AP: 0.9091  Sketch Index: 360


...

--------------------------- Class 95: guitar --------------------------- 
Min AP: 0.0134  Sketch Index: 7678 | Max AP: 0.6721  Sketch Index: 7669


--------------------------- Class 96: hamburger --------------------------- 
Min AP: 0.0125  Sketch Index: 7697 | Max AP: 0.8552  Sketch Index: 7704


--------------------------- Class 97: hammer --------------------------- 
Min AP: 0.0156  Sketch Index: 7801 | Max AP: 0.8095  Sketch Index: 7767


--------------------------- Class 98: hand --------------------------- 
Min AP: 0.0125  Sketch Index: 7871 | Max AP: 0.8967  Sketch Index: 7897


--------------------------- Class 99: harp --------------------------- 
Min AP: 0.0125  Sketch Index: 7936 | Max AP: 0.8272  Sketch Index: 7944


In [ ]:
print(classes_AP)

[[0.03818485217627361, 0.08639014699985789, 0.12938536325255073, 0.47316806711086096, 0.5695110324400633, 0.01436817190493661, 0.09354818335903863, 0.11636735120859258, 0.0648816442309593, 0.5494999718347969, 0.033507517823555565, 0.5506277592585633, 0.06796588693957116, 0.044375, 0.040930347080899696, 0.6709202346597024, 0.07809675265914029, 0.31967980999886936, 0.41490285360730733, 0.3881808286376666, 0.0696571742908313, 0.2957938967975639, 0.23587148805224264, 0.0125, 0.518128945154159, 0.17158144949857954, 0.09119753736934763, 0.039570914558361654, 0.4608555241276123, 0.05081168831168831, 0.10793625267114568, 0.5588657875043939, 0.5404916359269253, 0.5902283747928813, 0.3825796842647075, 0.12345053746706225, 0.4383935113202857, 0.541884575499648, 0.5339796825143169, 0.5233316920649851, 0.5735720155322653, 0.5713664133254396, 0.6095128619568009, 0.12765361419658552, 0.36105528153296484, 0.36176056563241266, 0.5326962857772995, 0.07249244034833943, 0.4143294641480317, 0.2810896508244

In [ ]:
# sort dict by decreasing value of mAP
from operator import itemgetter
classes_stats_s = sorted(classes_stats, key=itemgetter('label')) 

# save results on file 
import json
filepath = "/content/gdrive/Shareddrives/COMPUTER_VISION/analysis_by_class/mAP_by_class"
print(filepath)
with open(filepath, 'w') as fout:
  json.dump(classes_stats_s, fout, indent=4)


/content/gdrive/Shareddrives/COMPUTER_VISION/analysis_by_class/mAP_by_class
